In [2]:
!pip install kafka-python

  Using cached requests-2.28.1-py3-none-any.whl (62 kB)
  Using cached charset_normalizer-2.1.1-py3-none-any.whl (39 kB)
  Using cached idna-3.4-py3-none-any.whl (61 kB)
  Using cached urllib3-1.26.13-py2.py3-none-any.whl (140 kB)
     -------------------------------------- 155.3/155.3 kB 3.1 MB/s eta 0:00:00



[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
import requests
import json
import pandas as pd
from kafka.admin import KafkaAdminClient, NewTopic
from kafka import KafkaProducer
from json import dumps
import time


In [4]:
with open("teams_ids.json", "r") as f:
    f_data = json.load(f)
    f.close()
teams_num = len(f_data['team'])

#### create kafka topic

In [5]:
def create_kafka_topic(topic_name):
    admin_client = KafkaAdminClient(
        bootstrap_servers="localhost:29092", 
        client_id='test',
        api_version=(2,0,2)
    )

    topic_list = []
    topic_list.append(NewTopic(name=topic_name, num_partitions=1, replication_factor=1))
    admin_client.create_topics(new_topics=topic_list, validate_only=False)

#####  write fixtures to kafka 

In [4]:
create_kafka_topic('fixtures')

In [16]:
producer = KafkaProducer(bootstrap_servers=['localhost:29092'],
                         api_version=(2,0,2),
                         value_serializer=lambda x: 
                         dumps(x).encode('utf-8'))

URL = "https://v3.football.api-sports.io/fixtures"

headers = {
    "Content-Type": "application/json",
    "x-rapidapi-host": "v3.football.api-sports.io",
    "x-rapidapi-key": "d344e44df5d6ddd17d81d42ace86c412"
}

topic_name = "fixtures"

for team in range(1,teams_num+1):
    if (team % 10 == 0)and(team!=teams_num+1):
        time.sleep(60)
    data = get_fixtures(team)
    
    for fixture in range(len(data['response'])):
        producer.send(topic_name, data['response'][fixture])

In [14]:
def get_fixtures(team):
    team_id = f_data['team'][team-1]
    params = {
        "league": "39",
        "season": "2021",
        "team": team_id
    }

    response = requests.get(
        URL,
        params = params,
        headers = headers
    )
    
    data = response.json()
    
    return data

In [17]:
URL = "https://v3.football.api-sports.io/fixtures"

headers = {
    "Content-Type": "application/json",
    "x-rapidapi-host": "v3.football.api-sports.io",
    "x-rapidapi-key": "d344e44df5d6ddd17d81d42ace86c412"
}

params = {
    "league": "39",
    "season": "2021",
    "team": "50"
}

response = requests.get(
    URL,
    params = params,
    headers = headers
)

data = response.json()


In [8]:
print(data['response'][0])

IndexError: list index out of range